In [1]:
import numpy as np

In [2]:
def run(expression: str, X, Y=None, equivalent: str =None):
    assert isinstance(expression, str)
    print(f"X:{X.shape}\n{X}\n")
    if Y is not None:
        print(f"Y:{Y.shape}\n{Y}\n")

    Z = eval(expression)
    print(f"{expression}:{Z.shape}\n{Z}\n")
    
    if equivalent:
        assert isinstance(equivalent, str)
        print(f"Equivalent:{equivalent}:\n{eval(equivalent)}")

# Clarification

```einsum``` operations of numpy/tensorflow are **NOT the same** Einstein Summation.

### Einstein Summation does not have output subscript in the explicit mode

* Transpose: ```ij,jk->kj```
* Diagonal: ```ii->i```
* Element-wise multipication without summation: ```ij,ij->ij```
* Self summation: 
    * ```ij->``` which is the same with ```sum(X, axis=None)```
    * ```ij...->j...``` which is the same with ```sum(X, axis=0)```
    * ```...ij->...i``` which is the same with ```sum(X, axis=-1)```

### Einstein Summation does not have transpose

* Transpose ```ji``` as in ```einsum("ji", X)```

### Einstein Summation is supposed to have dummy index (common subscript)
* Outper product with no common subscript: ```ij,kl```

## Coding Standard

To make the intention of using the **Einstein Summation** clear, do not use output subscript when the operation is that of original Einstein Summation.

---

# Einstein summation

For matrices $a_{(i,j)}$  and $b_{(j,k)}$, a common subscript $j$ specifies the summation axis along which to add the elementwise multiplications $a_{i j} * b_{j k}$. This summation notation $a_{i j} b_{j k}$ is called Einstein summation. The dimension size ```J``` along the axis ```j``` matches in the shapes of ```a:(I,J)``` and ```b:(J,K)```.


$
\begin{align*}
a_{i j} b_{j k}
&= \sum\limits ^J_{j} a_{ij} * b_{jk}
\\
&=  a_{i 1} * b_{1 k} + a_{i 2} * b_{2 k} + \dots + a_{i j} * b_{j k} + \dots + a_{i J} * b_{j J}
\end{align*}
$



* [Einstein Summation](https://mathworld.wolfram.com/EinsteinSummation.html)

>Einstein summation is a notational convention for simplifying expressions including summations of vectors, matrices, and general tensors. There are essentially three rules of Einstein summation notation, namely:
>1. Repeated indices are implicitly summed over.
>2. Each index can appear at most twice in any term.
>3. Each term must contain identical non-repeated indices.




For ```np.einsum("ij,j", a, b)``` of the **green rectangle** in the diagram from the youtube, ```j``` is the dummy index and the element wise multiplication ```a[i][j] * b[j]``` is summed up along the ```j``` axis as $ \sum\limits_{j} (a_{ij} * b_j) $ . 


* [Einstein Summation Convention: an Introduction](https://www.youtube.com/watch?v=CLrTj7D2fLM)


<img src="images/einstein_summation.png" align="left">

In [3]:
a = np.arange(6).reshape((2,3))
b = np.arange(10,14).reshape((2,2))
print(f"a:{a}\n")
print(f"b:{b}\n")

a:[[0 1 2]
 [3 4 5]]

b:[[10 11]
 [12 13]]



## $np.einsum(\text{"ij,il"}, a, b)$

For the dummy index $i$, which is common in $a_{ik}$ and $ b_{il}$,  ```np.einsum(a, b)``` appies the operation $a_{i k} b_{i l} $ for all the (k, l) combinations.

$
\begin{align*}
a_{i k} b_{i l} &= a_{0 k} * b_{0 l} + a_{1 k} * b_{1l} 
\\
&= \sum\limits _{i} a_{ik} * b_{il}
\\
&= a^T[k] \cdot b^T[l]
\end{align*}
$

The dummy index can appear anywhere as long as the rules (please see the youtube for details) are met. For the dummy index ```i``` in ```np.einsum(“ik,il", a, b)```, it is a row index of the matrices ```a``` and ```b```, hence a column from ```a```  and that from ```b``` are extracted to generate the **dot product**s. 

<img src="images/einsum_operation.png" align="left">

# Approach to apply einsum

The basis of the Einstein Summation is ```dot product``` $\sum\limits_j x_{ij} * y_{kj}$ with two rows ```X[i]``` and ```Y[k]```.

1. Layout the output ```Z``` shape ```(i,k)```. 
2. Frame two windows to identify ```X[i]``` and ```Y[k]``` (for each free indices ```i``` and ```k```) along the dummy index ```j``` .
3. Apply ```dot product``` on the two rows, and fill ```Z[i][k]```.
4. Slide one window ```k``` with another ```i``` being fixed.

<img src="images/np_einsum.JPG" align="left"/>

## Matmul $X^T @ Y$ as ```"ij,ik"``` of shape ```(j,k)```

* ```X:(i, j)```
* ```Y:(i, k)``` 

```Z:(j,k)``` == ```X.T:(j,i)@Y:(i,k)``` == ```"ij,ik->jk"```

In [4]:
X = np.arange(1, 7).reshape((3,2)).T
Y = np.arange(7, 15).reshape((4,2)).T

expression = "np.einsum('ij,ik->jk', X, Y)"
equivalent = "np.matmul(X.T, Y)"
run(expression=expression, X=X, Y=Y, equivalent=equivalent)

X:(2, 3)
[[1 3 5]
 [2 4 6]]

Y:(2, 4)
[[ 7  9 11 13]
 [ 8 10 12 14]]

np.einsum('ij,ik->jk', X, Y):(3, 4)
[[ 23  29  35  41]
 [ 53  67  81  95]
 [ 83 105 127 149]]

Equivalent:np.matmul(X.T, Y):
[[ 23  29  35  41]
 [ 53  67  81  95]
 [ 83 105 127 149]]


## Matmul $X @ Y^T$ as ```'ij,kj'``` of shape ```(i,k)```

* ```X:(i, j)```
* ```Y:(k, j)``` 

```Z:(i,k)``` == ```X:(i,j)@Y.T:(j,k)``` == ```ij,kj```


In [5]:
X = np.arange(1, 7).reshape((3,2))
Y = np.arange(7, 15).reshape((4,2))

expression = "np.einsum('ij,kj', X, Y)"
equivalent = "np.matmul(X, Y.T)"
run(expression=expression, X=X, Y=Y, equivalent=equivalent)

X:(3, 2)
[[1 2]
 [3 4]
 [5 6]]

Y:(4, 2)
[[ 7  8]
 [ 9 10]
 [11 12]
 [13 14]]

np.einsum('ij,kj', X, Y):(3, 4)
[[ 23  29  35  41]
 [ 53  67  81  95]
 [ 83 105 127 149]]

Equivalent:np.matmul(X, Y.T):
[[ 23  29  35  41]
 [ 53  67  81  95]
 [ 83 105 127 149]]


# Tensordot

* ```tensordot(X, Y, axes=([x-dummy-axes], [y-dummy-axes]))```

Instead of the common subscript ```j``` in ```np.einsum('ij,kj', X, Y)```, specify where the dummy indices are with **axes**.

* [Numpys `tensordot` and what is happening mathematically](https://scicomp.stackexchange.com/a/34720/38855)

> axes[0] and axes[1] specify the locations of the dumy indices in the parameters X, Y of einsum. <br><br>
>```np.tensordot(a, b, axes=[(0,2),(3,1)])``` corresponds to ```np.einsum('ijkl,mkni', a, b)``` <br>where ```'ijkl'[(0,2)] == 'ik' == 'mkni'[(3,1)]```.




In [6]:
X = np.arange(1, 7).reshape((3,2))
Y = np.arange(7, 15).reshape((4,2))

# The 1-th axis in X and 1-th axis in Y are the dummy index j as in "einsum('ij,kj', X, Y)"
expression = "np.tensordot(X, Y, axes=([1],[1]))"
equivalent = "np.einsum('ij,kj', X, Y)"
run(expression=expression, X=X, Y=Y, equivalent=equivalent)

X:(3, 2)
[[1 2]
 [3 4]
 [5 6]]

Y:(4, 2)
[[ 7  8]
 [ 9 10]
 [11 12]
 [13 14]]

np.tensordot(X, Y, axes=([1],[1])):(3, 4)
[[ 23  29  35  41]
 [ 53  67  81  95]
 [ 83 105 127 149]]

Equivalent:np.einsum('ij,kj', X, Y):
[[ 23  29  35  41]
 [ 53  67  81  95]
 [ 83 105 127 149]]


### Multiple dummy indices

Apply $\sum\limits_j \sum\limits_j X_{jk} * Y_{hijk} $ for each ```(k, j)``` elements to create shape ```(h,i)``` result.

<img src="images/np_tensor_dot_multi_dummy_indices.png" align="left"/>

In [7]:
X = np.arange(6).reshape(3,2)
Y = np.arange(24).reshape(2,2,2,3)

# --------------------------------------------------------------------------------
# 'kj,hijk' with two dummy idices 'k' and 'j'.
# 'k' and 'j' are dropped from (h,i,j,k) and result in shape:(h,i)/
# --------------------------------------------------------------------------------
equivalent = "np.einsum('kj,hijk', X, Y)"

# --------------------------------------------------------------------------------
# Dummy idices in X is [0th, 1st] axes, corresponding to [3rd, 2nd] axes in Y
# --------------------------------------------------------------------------------
expression = "np.tensordot(X, Y, axes=([0,1],[3,2]))"
run(expression=expression, X=X, Y=Y, equivalent=equivalent)

X:(3, 2)
[[0 1]
 [2 3]
 [4 5]]

Y:(2, 2, 2, 3)
[[[[ 0  1  2]
   [ 3  4  5]]

  [[ 6  7  8]
   [ 9 10 11]]]


 [[[12 13 14]
   [15 16 17]]

  [[18 19 20]
   [21 22 23]]]]

np.tensordot(X, Y, axes=([0,1],[3,2])):(2, 2)
[[ 50 140]
 [230 320]]

Equivalent:np.einsum('kj,hijk', X, Y):
[[ 50 140]
 [230 320]]


---

# einsum proprietary extentions

## Output subscript

```einsum``` proprietary transformation can be specified ```np.einsum("... -> <transformation>")``` with the **output subscript labels** with ```->``` identifier.

* Transpose
* Diagonal
* Outer product
* Self summation with the axis to sum along by ommiting the subscript.
* Element-wise multiplication without summation
* Elipsis

See the **explicit mode** in [numpy.einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) for details.

>In explicit mode the output can be directly controlled by specifying output subscript labels. This requires the identifier ```‘->’``` as well as the list of output subscript labels. This feature increases the flexibility of the function since summing can be disabled or forced when required. The call ```np.einsum('i-', a)``` is like ```np.sum(a,axis=-1)```, and ```np.einsum('ii->i', a)``` is like ```np.diag(a)```. <br>
The difference
is that einsum does not allow broadcasting by default. Additionally
```np.einsum('ij,jh->ih', a, b)``` directly specifies the order of the
output subscript labels and therefore returns matrix multiplication,
unlike the example above in implicit mode.


For a, i is dummy index and row index. Hence it extracts column j (all i values)
For b, i is dummy index and row index. Hence it extracts column l
Then dot product of the columns are generated.
e.g. 
```
a[i=0,1][j=0] from a -> [0,3]
b[i=0,1][l=0] from b -> [10, 12]
36 = np.dot([0,3], [10, 12])
```
The result shape is (3,2) because 
```
a[i=*][j=0,1,2] -> 3
b[i=*][l=0,1]   -> 2 
```
c = np.einsum("ij,il", a, b)    # Shape (3,2)

In [8]:
print(f"a:(i,k):\n{a}\n")
print(f"b:(k,l)\n{b}\n")
print(f"(ik,il):\n{np.einsum('ik,il', a, b)}")

a:(i,k):
[[0 1 2]
 [3 4 5]]

b:(k,l)
[[10 11]
 [12 13]]

(ik,il):
[[36 39]
 [58 63]
 [80 87]]


## No dummy index (Cartesian outer product)

No summation.
1. A term (subscript Indices, e.g. "ij") selects an element in each array.
2. Each left-hand side element is applied on the element on the right-hand side for element-wise multiplication (hence multiplication always happens).

```a``` has shape (2,3) each element of which is applied to ```b``` of shape (2,2). Hence it creates a matrix of shape ```(2,3,2,2)``` without no summation as ```(i,j)```, ```(k.l)``` are all free indices.

In [9]:
# --------------------------------------------------------------------------------
# For np.einsum("ij,kl", a, b)
# 1-1: Term "ij" or (i,j), two free indices, selects selects an element a[i][j].
# 1-2: Term "kl" or (k,l), two free indices, selects selects an element b[k][l].
# 2:   Each a[i][j] is applied on b[k][l] for element-wise multiplication a[i][j] * b[k,l]
# --------------------------------------------------------------------------------
# for (i,j) in a:
#    for(k,l) in b:
#        a[i][j] * b[k][l]
np.einsum("ij,kl", a, b)

array([[[[ 0,  0],
         [ 0,  0]],

        [[10, 11],
         [12, 13]],

        [[20, 22],
         [24, 26]]],


       [[[30, 33],
         [36, 39]],

        [[40, 44],
         [48, 52]],

        [[50, 55],
         [60, 65]]]])

---

# transpose

## Transpose $X \rightarrow X^T$ as ```"ij->ji"```


In [10]:
X = np.arange(6).reshape(2,3)

expression = "np.einsum('ij->ji', X)"
equivalent = "np.transpose(X)"
run(expression=expression, X=X, equivalent=equivalent)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

np.einsum('ij->ji', X):(3, 2)
[[0 3]
 [1 4]
 [2 5]]

Equivalent:np.transpose(X):
[[0 3]
 [1 4]
 [2 5]]


### Abbrebiation ```"ji"```

```np.einsum``` detects the **alphabetical order inversion** and apply axis swap(s).

As per the coding standard, do NOT use this format.

In [11]:
X = np.arange(6).reshape(2,3)
expression = "np.einsum('ji', X)"
quivalent = "np.transpose(X)"
run(expression=expression, X=X, equivalent=equivalent)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

np.einsum('ji', X):(3, 2)
[[0 3]
 [1 4]
 [2 5]]

Equivalent:np.transpose(X):
[[0 3]
 [1 4]
 [2 5]]


## Tranapose of inner matrix as ```"inm->imn"```

In [12]:
X = np.arange(12).reshape((2, 2, 3))
expression = "np.einsum('inm->imn', X)"
run(expression=expression, X=X)

X:(2, 2, 3)
[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]

np.einsum('inm->imn', X):(2, 3, 2)
[[[ 0  3]
  [ 1  4]
  [ 2  5]]

 [[ 6  9]
  [ 7 10]
  [ 8 11]]]



## dot prducts of Matrix A rows and Matrix B columns


<img src="images/dot_proucts_from_matrix_A_rows_and_B_rows.png" align="left" />

In [13]:
import numpy as np
A = np.matrix('0 1 2; 3 4 5')
B = np.matrix('0 -3; -1 -4; -2 -5');

fmt="np.einsum('ij,ji->i', A, B)"
print(f"{fmt} = {np.einsum('ij,ji->i', A, B)}")

fmt="np.diagonal(np.matmul(A,B))"
print(f"{fmt} = {np.diagonal(np.matmul(A,B))}")

fmt="(A*B).diagonal()"
print(f"{fmt} = {(A*B).diagonal()}")

np.einsum('ij,ji->i', A, B) = [ -5 -50]
np.diagonal(np.matmul(A,B)) = [ -5 -50]
(A*B).diagonal() = [[ -5 -50]]


---
# diagonal as ```"ii->i"```

This is numpy proprietary expression as **```"ii"``` is illegal** for the Einstein Summation script.

In [14]:
X = np.arange(9).reshape((3,3))
expression = "np.einsum('ii->i', X)"
equivalent = "np.diag(X)"
run(expression=expression, X=X, equivalent=equivalent)

X:(3, 3)
[[0 1 2]
 [3 4 5]
 [6 7 8]]

np.einsum('ii->i', X):(3,)
[0 4 8]

Equivalent:np.diag(X):
[0 4 8]


---
# sum

## Sum(X) as ```"ij->"```

In [15]:
X = np.arange(6).reshape((2,3))
print(f"X:{X.shape}\n{X}\n")

expression = "np.einsum('ij->', X)"
equivalent = "np.sum(X)"
run(expression=expression, X=X, equivalent=equivalent)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

X:(2, 3)
[[0 1 2]
 [3 4 5]]

np.einsum('ij->', X):()
15

Equivalent:np.sum(X):
15


## Sum(X, axis=0) as ```"ij...->j..."```

The axis **omitted** is the **axis to sum** along. The first axis ```i``` is omitted from ```j...```, hence ```sum(axis=0)```.

In [16]:
X = np.arange(6).reshape((2,3))
print(f"X:{X.shape}\n{X}\n")

expression = "np.einsum('ij...->j...', X)"
equivalent = "np.sum(X, axis=0)"
run(expression=expression, X=X, equivalent=equivalent)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

X:(2, 3)
[[0 1 2]
 [3 4 5]]

np.einsum('ij...->j...', X):(3,)
[3 5 7]

Equivalent:np.sum(X, axis=0):
[3 5 7]


## Sum(X, axis=-1) as ```"...ij->...i"```

The axis **omitted** is the **axis to sum** along. The last axis ```j``` is omitted in the output subscript ```...i```, hence ```sum(axis=-1)```.

In [17]:
X = np.arange(6).reshape((2,3))
print(f"X:{X.shape}\n{X}\n")

expression = "np.einsum('...ij->i', X)"
equivalent = "np.sum(X, axis=1)"
run(expression=expression, X=X, equivalent=equivalent)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

X:(2, 3)
[[0 1 2]
 [3 4 5]]

np.einsum('...ij->i', X):(2,)
[ 3 12]

Equivalent:np.sum(X, axis=1):
[ 3 12]


---
# Element-wise multiplication

When the left subscript is the same with result, ```ensum``` performs the element-wise operation.

$
\begin {align*}
i,j \rightarrow i
\end {align*}
$ where the left subscript ```i``` is the same with output subscript ```i``` performs $x_i + y_i$.

$
\begin {align*}
ij,ij \rightarrow ij
\end {align*}
$ where the left subscript ```ij``` is the same with output subscript ```ij``` performs $x_{ij} + y_{ij}$.

In [18]:
X = np.arange(6).reshape((2,3))
Y = np.arange(7, 13).reshape((2,3))
expression = "np.einsum('ij,ij->ij', X, Y)"
equivalent = "np.multiply(X, Y)"

run(expression=expression, X=X, Y=Y, equivalent=equivalent)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

Y:(2, 3)
[[ 7  8  9]
 [10 11 12]]

np.einsum('ij,ij->ij', X, Y):(2, 3)
[[ 0  8 18]
 [30 44 60]]

Equivalent:np.multiply(X, Y):
[[ 0  8 18]
 [30 44 60]]


---
# Product

* dot product
* outer product
* matmul

## dot as ```'i,i'``` or ```ij,ij```

This is the Einstein Summation.

For all ```i```, **multiply** $x_{i}$ on $y_{i}$ and **sum** them up. NOTE that ```dot``` is 1 dimensional and ```np.dot``` is NOT exactly the dot operation hence use ```np.inner```.

$
\begin {align*}
X \cdot Y = \sum\limits_j x_{i} * y_{i}
\end {align*}
$

NOTE that with ```einsum (Not Einstein Summation)``` the left subscript **```i```** (or **```ij```** ) is the same with the right subscript **```i```** (or **```ij```** ) for ***inner*** operations.

In [19]:
X = np.arange(3)
Y = np.arange(3, 6)
expression = "np.einsum('i,i', X, Y)"
equivalent = "np.inner(X, Y)"
run(expression=expression, X=X, Y=Y, equivalent=equivalent)

X:(3,)
[0 1 2]

Y:(3,)
[3 4 5]

np.einsum('i,i', X, Y):()
14

Equivalent:np.inner(X, Y):
14


### sum of row dot products 

1. For each row ```i```: multiply $x_{ij}$ with $y_{ij}$ for all ```j``` and sum them up. 

$
\begin {align*}
\qquad z_{i} = 
\begin {bmatrix}
\sum\limits_j x_{ij} * y_{ij}
\end {bmatrix}
\end {align*}
$

2. Sum the dot products

$
\begin {align*}
\qquad \sum\limits_i z_{i}
\end {align*}
$

Note that **there is NO matrix equivalent** operation for this ```einsum``` operation.

This is the same with:

$
\begin {align*}
\sum\limits_i \sum\limits_j x_{ij} * y_{ij}
\end {align*}
$


In [20]:
X = np.arange(6).reshape((2,3))
Y = np.arange(7, 13).reshape((2,3))
expression = "np.einsum('ij,ij', X, Y)"
run(expression=expression, X=X, Y=Y)

X:(2, 3)
[[0 1 2]
 [3 4 5]]

Y:(2, 3)
[[ 7  8  9]
 [10 11 12]]

np.einsum('ij,ij', X, Y):()
160



## dot product (N,1) from X:(N,D) and Y(N,D) as ```'nd,nd->n'```

Stack dot product of $X_i$, $Y_i$

In [22]:
N = 3
D = 2
size = N*D
shape = (N,D)
X = np.arange(size).reshape(shape)
Y = np.arange(size, 2*size).reshape(shape)
expression = "np.einsum('nd,nd->n', X, Y)"
run(expression=expression, X=X, Y=Y)

X:(3, 2)
[[0 1]
 [2 3]
 [4 5]]

Y:(3, 2)
[[ 6  7]
 [ 8  9]
 [10 11]]

np.einsum('nd,nd->n', X, Y):(3,)
[ 7 43 95]



## Outer product

For each ```i``` in ```X```, element-wise multiply $x_i$ on all $y_k$. 

$
\begin {align*}
X \otimes Y = x_i * y_k
\end {align*}
$

NOTE that the **left** subscript **```i```** (or **```ij```** ) is different from the **right** subscript **```k```** (or **```kl```** ) for ***outer*** operations.

### 1 dimensional outer product as ```'i,j->ij'```

In [ ]:
X = np.arange(3)
Y = np.arange(3, 6)
expression = "np.einsum('i,j->ij', X, Y)"
run(expression=expression, X=X, Y=Y)

### 2 dimensions **per-row** outer product as ```'ij,kl->ij'```

This is NOT cartesian produdt but **per-row** outer product (extention of ```i,j->ij```).

In [ ]:
X = np.arange(6).reshape((2,3))
Y = np.arange(7, 13).reshape((2,3))
expression = "np.einsum('ij,kl->ij', X, Y)"
run(expression=expression, X=X, Y=Y)

### Cartesian outer product as ```'ij,kl->ijkl'``` or ```'ij,kl'```

In [ ]:
X = np.arange(6).reshape((2,3))
Y = np.arange(7, 13).reshape((2,3))
expression = "np.einsum('ij,kl->ijkl', X, Y)"
run(expression=expression, X=X, Y=Y)

## Matmul $X @ Y$ as ```"ij,jk->ik"```


* ```X:(i, j)```
* ```Y:(j, k)``` 

```Z:(i,k)``` == ```X.T:(i,j)@Y:(j,k)``` == ```"ij,jk->jk"```

In [ ]:
X = np.arange(1, 7).reshape((3,2))
Y = np.arange(7, 15).reshape((2,4))

expression = "np.einsum('ij,jk->ik', X, Y)"
equivalent = "np.matmul(X, Y)"
run(expression=expression, X=X, Y=Y, equivalent=equivalent)

---

---